In [ ]:
%load_ext autoreload
%autoreload 2

from mimic_iv_analysis.io import DataLoader, TableNamesHOSP, TableNamesICU, ParquetConverter, ExampleDataLoader
import pandas as pd
import dask.dataframe as dd

In [ ]:
from dask.distributed import Client
client = Client(n_workers=4, threads_per_worker=4, processes=True, memory_limit='8GB')
client

2025-05-16 11:09:55,142 - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2025-05-16 11:09:55,153 - INFO - State start
2025-05-16 11:09:55,154 - INFO - Found stale lock file and directory '/var/folders/yr/5jld3pmj3p75vsk6f7dsr6gm0000gn/T/dask-scratch-space/scheduler-_qpplges', purging
2025-05-16 11:09:55,154 - INFO - Found stale lock file and directory '/var/folders/yr/5jld3pmj3p75vsk6f7dsr6gm0000gn/T/dask-scratch-space/scheduler-pobsm7ir', purging
2025-05-16 11:09:55,157 - INFO -   Scheduler at:     tcp://127.0.0.1:62477
2025-05-16 11:09:55,157 - INFO -   dashboard at:  http://127.0.0.1:8787/status
2025-05-16 11:09:55,157 - INFO - Registering Worker plugin shuffle
2025-05-16 11:09:55,166 - INFO -         Start Nanny at: 'tcp://127.0.0.1:62480'
2025-05-16 11:09:55,177 - INFO -         Start Nanny at: 'tcp://127.0.0.1:62482'
2025-05-16 11:09:55,185 - INFO -         Start Nanny at: 'tcp://127.0.0.1:62484'
2

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 29.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62477,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:62490,Total threads: 4
Dashboard: http://127.0.0.1:62492/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:62480,


2025-05-16 11:10:02,349 - WARNING - Shuffle 2281788b33f0ccaae3c0c85e35664272 initialized by task ('shuffle-transfer-2281788b33f0ccaae3c0c85e35664272', 0) executed on worker tcp://127.0.0.1:62490
2025-05-16 11:10:02,908 - WARNING - Shuffle 2281788b33f0ccaae3c0c85e35664272 deactivated due to stimulus 'task-finished-1747408202.907464'
2025-05-16 11:10:11,896 - WARNING - Shuffle b26dc4254b3b64602a83c446fdbdf3cf initialized by task ('shuffle-transfer-b26dc4254b3b64602a83c446fdbdf3cf', 0) executed on worker tcp://127.0.0.1:62490
2025-05-16 11:10:11,924 - WARNING - Shuffle b26dc4254b3b64602a83c446fdbdf3cf deactivated due to stimulus 'task-finished-1747408211.923368'
2025-05-16 11:10:12,038 - WARNING - Shuffle 2281788b33f0ccaae3c0c85e35664272 initialized by task ('shuffle-transfer-2281788b33f0ccaae3c0c85e35664272', 0) executed on worker tcp://127.0.0.1:62490
2025-05-16 11:10:12,062 - WARNING - Shuffle 2281788b33f0ccaae3c0c85e35664272 deactivated due to stimulus 'task-finished-1747408212.061772

## Scan the directory

In [32]:
examples_full    = Examples(partial_loading=False)
examples_partial = Examples(partial_loading=True, num_subjects=20, random_selection=False)

2025-05-16 11:33:56,521 - INFO - Loading ----- poe ----- table.
2025-05-16 11:33:56,545 - INFO - Loading full table: 52212109 rows.
2025-05-16 11:33:56,571 - INFO - Applied filters: 52212109 rows.
2025-05-16 11:33:56,572 - INFO - Loading ----- poe_detail ----- table.
2025-05-16 11:33:56,596 - INFO - Loading full table: 8504982 rows.
2025-05-16 11:33:56,621 - INFO - Applied filters: 8504982 rows.
2025-05-16 11:33:56,622 - INFO - Loading ----- diagnoses_icd ----- table.
2025-05-16 11:33:56,648 - INFO - Loading full table: 9547064 rows.
2025-05-16 11:33:56,814 - INFO - Applied filters: 5179551 rows.
2025-05-16 11:33:56,814 - INFO - Loading ----- admissions ----- table.
2025-05-16 11:33:56,840 - INFO - Loading full table: 546028 rows.
2025-05-16 11:33:57,040 - INFO - Applied filters: 534159 rows.
2025-05-16 11:33:57,040 - INFO - Loading ----- patients ----- table.
2025-05-16 11:33:57,071 - INFO - Loading full table: 364627 rows.
2025-05-16 11:33:57,132 - INFO - Applied filters: 316908 rows

In [ ]:
examples_full.study_table_info()

,module,table_name,file_path,file_size,display_name,suffix,columns_list
3,hosp,poe,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,1054299614,poe 1.1 GB,.parquet,"{order_provider_id, hadm_id, order_type, order..."
9,hosp,poe_detail,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,105773003,poe_detail 105.8 MB,.parquet,"{field_name, poe_seq, poe_id, subject_id, fiel..."
11,hosp,diagnoses_icd,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,75013787,diagnoses_icd 75.0 MB,.parquet,"{hadm_id, seq_num, icd_version, subject_id, ic..."
13,hosp,admissions,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,25736908,admissions 25.7 MB,.parquet,"{deathtime, language, discharge_location, hadm..."
20,hosp,patients,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,4582613,patients 4.6 MB,.parquet,"{gender, dod, anchor_age, anchor_year, anchor_..."
21,hosp,d_icd_diagnoses,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,2683277,d_icd_diagnoses 2.7 MB,.parquet,"{icd_code, icd_version, long_title}"


In [53]:
examples_partial.counter()

Table           | Rows       | Subject IDs
--------------- | ---------- | ----------
patients        | 18         | 18        
admissions      | 36         | 20        
diagnoses_icd   | 59         | 6         
poe             | 1,976      | 20        
poe_detail      | 284        | 20        


In [ ]:
# Print in a tabular format
examples_partial.n_rows_after_merge()

DataFrame       Count      DataFrame       Count      DataFrame       Count     
----------------------------------------------------------------------
df12            31         patients        18         admissions      36        
df34            59         diagnoses_icd   59         d_icd_diagnoses 97441     
poe_merged      2046       poe             1976       poe_detail      284       
merged_wo_poe   59         df34            59         df12            31        
merged_w_poe    2682       poe_merged      2046       merged_wo_poe   59        
